In [ ]:
import pyathena
import pandas as pd
import os

# Credenciais aws
aws_access_key_id = os.environ['AWS_ACCESS_KEY_ID']
aws_secret_access_key = os.environ['AWS_SECRET_ACCESS_KEY']

In [ ]:
# Directly by panda
athena_conn = pyathena.connect(aws_access_key_id=aws_access_key_id, 
                 aws_secret_access_key=aws_secret_access_key, 
                 s3_staging_dir='s3://aws-athena-query-results-XXXXXXXXXXXXXXX-us-east-1/', 
                 region_name='us-east-1') 


print('CONXÃO FEITA COM SUCESSO!!!')

In [ ]:
query = """
SELECT * 
FROM consumer_zone.escritorio_vendas
order by data desc
"""


df = pd.read_sql(query, athena_conn)

In [ ]:
df.head(10)

In [ ]:
# excluindo as linhas com o nome "PAULO" na segunda coluna
df = df.drop(df[df['data'] == '2023-04-01'].index)


In [ ]:
df['filial_venda'] = df['filial_venda'].replace('FILIAL - EXPORTAÃÃO', 'Filial - EXPORTAÇÃO')

In [ ]:
# removendo a coluna 'data_mes' do dataframe
cols = df.columns

In [ ]:

filtro = (df['data'] == '2023-01-01') 
df.loc[filtro, cols] = df.loc[df['data'] == '2023-03-01', cols].values
# exibindo o dataframe após a atualização
print(df)


In [ ]:
# Credenciais aws
BUCKET_NAME = 'XXXXXXXXXXXXXXXXX0336967'
AWS_ACCESS_KEY_ID = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = os.environ['AWS_SECRET_ACCESS_KEY'] 
import os, io

import boto3

s3_client=boto3.client(
    's3', 
    region_name='us-east-1',
    aws_access_key_id=AWS_ACCESS_KEY_ID, 
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY 
)

key_parquet_consumer = f'consumer-zone/XXXXXXXXXXXXXXX/escritorio_venda/fat/historico.parquet'
key_manifest = f'consumer-zone/XXXXXXXXXXXXXXX/escritorio_venda/fat/_symlink_format_manifest/manifest_historico'


In [ ]:
out_buffer = io.BytesIO() # criando espaço na memória
df.to_parquet(out_buffer, index=False) # salvando na memória em formato parquet
s3_client.put_object(Bucket=BUCKET_NAME, Key=key_parquet_consumer, Body=out_buffer.getvalue())

In [ ]:
manifest_contents_new = f's3://{BUCKET_NAME}/{key_parquet_consumer}\n'

s3_client.put_object(
    Body=manifest_contents_new,
    Bucket=BUCKET_NAME,
    Key=key_manifest,
)